<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Lab: Fun with Neural Nets

---

Below is a procedure for building a neural network to recognize handwritten digits.  The data is from [Kaggle](https://www.kaggle.com/c/digit-recognizer/data), and you will submit your results to Kaggle to test how well you did!

1. Load the training data (`train.csv`) from [Kaggle](https://www.kaggle.com/c/digit-recognizer/data)
2. Setup X and y (feature matrix and target vector)
3. Split X and y into train and test subsets.
4. Preprocess your data

   - When dealing with image data, you need to normalize your `X` by dividing each value by the max value of a pixel (255).
   - Since this is a multiclass classification problem, keras needs `y` to be a one-hot encoded matrix
   
5. Create your network.

   - Remember that for multi-class classification you need a softmax activation function on the output layer.
   - You may want to consider using regularization or dropout to improve performance.
   
6. Trian your network.
7. If you are unhappy with your model performance, try to tighten up your model by adding hidden layers, adding hidden layer units, chaning the activation functions on the hidden layers, etc.
8. Load in [Kaggle's](https://www.kaggle.com/c/digit-recognizer/data) `test.csv`
9. Create your predictions (these should be numbers in the range 0-9).
10. Save your predictions and submit them to Kaggle.

---

For this lab, you should complete the above sequence of steps for _at least_ two of the four "configurations":

1. Using a `tensorflow` network (we did _not_ cover this in class!)
2. Using a `keras` convolutional network
3. Using a `keras` network with regularization
4. Using a `tensorflow` convolutional network (we did _not_ cover this in class!)

In [10]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras import utils as np_utils
from sklearn.model_selection import train_test_split

### Load training data

Load `train.csv` from Kaggle into a pandas DataFrame.

In [37]:
train = pd.read_csv('train.csv')

### Set up X and y

NOTE: Keras requires a `numpy` matrix, it doesn't work with `pandas`.

In [38]:
X = train[train.columns[1:]].values
y = train['label']

### Preprocessing

1. When dealing with image data, you need to normalize your `X` by dividing each value by the max number of pixels (255).
2. Since this is a multiclass classification problem, keras needs `y` to be a one-hot encoded matrix

In [40]:
X = X / 255.
y = np_utils.to_categorical(y)

### Train/Test Split

We want to create a validation set that the model will never see to approximate how it's going to do with Kaggle's `test.csv`. Use `sklearn`'s `train_test_split` to do this.

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2003)

### Create your neural network

Create a neural network using the `Dense` and `Dropout` layers from `keras`. Your activation function for the final output layer needs to be `softmax` to accomidate the ten different classes.

In [52]:
model = Sequential()
model.add(Dense(X_train.shape[1], input_shape=(784,), activation='relu'))
model.add(Dropout(.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

### Compile your model

Since this is a multiclass classification problem, your loss function is `categorical_crossentropy`.

In [44]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

### Fit the model

Use your X_test, y_test from the `train_test_split` step for the `validation_data` parameter.

In [45]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=100)

Train on 31500 samples, validate on 10500 samples
Epoch 1/5
31500/31500 [==============================] - 8s 239us/step - loss: 0.3045 - acc: 0.9112 - val_loss: 0.1700 - val_acc: 0.9510
Epoch 2/5
31500/31500 [==============================] - 7s 218us/step - loss: 0.1254 - acc: 0.9636 - val_loss: 0.1294 - val_acc: 0.9618
Epoch 3/5
31500/31500 [==============================] - 7s 219us/step - loss: 0.0786 - acc: 0.9767 - val_loss: 0.1091 - val_acc: 0.9679
Epoch 4/5
31500/31500 [==============================] - 7s 223us/step - loss: 0.0523 - acc: 0.9851 - val_loss: 0.0931 - val_acc: 0.9717
Epoch 5/5
31500/31500 [==============================] - 7s 237us/step - loss: 0.0367 - acc: 0.9899 - val_loss: 0.0886 - val_acc: 0.9745


### Load in Kaggle's `test.csv`

Be sure to do the **same** preprocessing you did for your training `X`.

In [47]:
test = pd.read_csv('test.csv')
test = test / 255.

### Create your predictions

Use `predict_classes` to get the actual numerical values (0-9).

In [48]:
pred = model.predict_classes(test.values)

### Prepare your submission

1. Add your predictions to a column called `Label`
2. You'll need to manually create the `ImageId` column, which is just a list of 1..[NUMBER OF TEST SAMPLES]

In [50]:
test['Label'] = pred
test['ImageId'] = range(1,test.shape[0] + 1)

### Create your submission csv

Remember to set `index=False`!

In [51]:
test[['ImageId', 'Label']].to_csv('submission.csv', index=False)